In [0]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize
import skimage
import cv2

In [0]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    name = ['Genuine','Fake']
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = skimage.io.imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
#             print(flat_data)
            images.append(img_resized)
            target.append(name[i])
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)
#     print(images[1])
#     print(images[[0]])
    print(target)
    # print(flat_data[[1]].shape)
    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [52]:
image_dataset = load_image_files("/content/drive/My Drive/Colab Notebooks/SVM-Image-Classification/All notes/")

['Genuine' 'Genuine' 'Genuine' 'Genuine' 'Genuine' 'Genuine' 'Genuine'
 'Genuine' 'Genuine' 'Genuine' 'Genuine' 'Genuine' 'Genuine' 'Fake' 'Fake'
 'Fake' 'Fake' 'Fake' 'Fake' 'Fake' 'Fake']


In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

In [55]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [56]:
y_pred = clf.predict(X_test)
y_pred

array(['Fake', 'Fake', 'Genuine', 'Genuine', 'Genuine', 'Genuine',
       'Genuine'], dtype='<U7')

In [57]:
X_test[[0]].shape

(1, 12288)

In [0]:
def preprocessing(image):
    image = resize(image,(64,64), anti_aliasing=True, mode='reflect')
    b = new.flatten()
    img = b.reshape(1,-1)
    return img

In [0]:
a = cv2.imread('/content/drive/My Drive/Colab Notebooks/SVM-Image-Classification/All notes/Know.jpg')
testimage = preprocessing(a)

In [60]:
clf.predict(testimage)

array(['Genuine'], dtype='<U7')

In [61]:
print("Classification report for:\n{}".format(metrics.classification_report(y_test, y_pred)))

Classification report for:
              precision    recall  f1-score   support

        Fake       1.00      0.50      0.67         4
     Genuine       0.60      1.00      0.75         3

    accuracy                           0.71         7
   macro avg       0.80      0.75      0.71         7
weighted avg       0.83      0.71      0.70         7

